In [1]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

In [3]:
try:
    _ = first_run
except NameError:
    first_run = True
    os.chdir(os.getcwd().rsplit("/", 1)[0])
    from _aux import features as F

# Load Data

Now that we are ready to present our final results, we must load the test set which we held out at step "0_split_data".

In [10]:
X_train, y_train = joblib.load("../data/train/preprocessed/train_features_labels.joblib.gz")

X_test, y_test = pd.read_csv("../data/test/X_test.csv", index_col=0), pd.read_csv("../data/test/y_test.csv", index_col=0)

# Load the preprocessor

In [5]:
preprocessor = joblib.load("../ml_artifacts/preprocessor.joblib.gz")

# Fit the best model

In [37]:
model = RandomForestRegressor().fit(X_train, y_train)

_ = joblib.dump(baseline, "../ml_artifacts/baseline_model.joblib.gz")

In [49]:
threshold_perf = pd.DataFrame(
    [
        (threshold, *confusion_matrix(y_validation, (baseline.predict(X_validation) > threshold).astype(int)).ravel())
        for threshold in np.arange(.05, .95, .05)
    ],
    columns=["threshold", "tn", "fp", "fn", "tp"]
).assign(
    precision=lambda df: df["tp"] / (df["tp"] + df["fp"]),
    recall=lambda df: df["tp"] / (df["tp"] + df["fn"]),
    f1=lambda df: 2 * (df["precision"] * df["recall"]) / (df["precision"] + df["recall"])
)

threshold_perf.to_csv("../ml_artifacts/baseline_model_performance.csv", index=False)

In [47]:
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)


threshold_perf.style.apply(
    highlight_max, color='darkorange', subset=["precision", "recall", 'f1']
)

,threshold,tn,fp,fn,tp,precision,recall,f1
0,0.050000,13582,600,128,86,0.125364,0.401869,0.191111
1,0.100000,13830,352,143,71,0.167849,0.331776,0.222920
2,0.150000,13930,252,160,54,0.176471,0.252336,0.207692
3,0.200000,13992,190,168,46,0.194915,0.214953,0.204444
4,0.250000,14032,150,172,42,0.218750,0.196262,0.206897
5,0.300000,14069,113,178,36,0.241611,0.168224,0.198347
6,0.350000,14097,85,184,30,0.260870,0.140187,0.182371
7,0.400000,14117,65,189,25,0.277778,0.116822,0.164474
8,0.450000,14134,48,197,17,0.261538,0.079439,0.121864
9,0.500000,14145,37,200,14,0.274510,0.065421,0.105660
